In [1]:
# 🛠️ FEATURE ENGINEERING - INITIALIZATION
print("\n6. 🛠️ FEATURE ENGINEERING")
print("=" * 50)

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
import os

print("🚀 Starting feature engineering pipeline...")

# Try multiple paths to find the sentiment analysis data
possible_paths = [
    'notebook/data/processed/sentiment_analysis_results.csv',
    '../notebook/data/processed/sentiment_analysis_results.csv',
    '../../notebook/data/processed/sentiment_analysis_results.csv',
    './data/processed/sentiment_analysis_results.csv',
    '../data/processed/sentiment_analysis_results.csv',
    'sentiment_analysis_results.csv'
]

sentiment_df = None
for path in possible_paths:
    try:
        if os.path.exists(path):
            sentiment_df = pd.read_csv(path)
            print(f"✅ sentiment_df loaded from: {path}")
            break
        else:
            print(f"❌ Not found: {path}")
    except Exception as e:
        print(f"⚠️ Error reading {path}: {str(e)}")

if sentiment_df is not None:
    # Create feature_df from sentiment_df
    feature_df = sentiment_df.copy()
    print(f"📊 Original dataset shape: {feature_df.shape}")
    print("✅ Feature DataFrame initialized")
    
    # Display basic info
    print(f"\n🔍 SENTIMENT_DF INFO:")
    print(f"   • Columns: {list(sentiment_df.columns)}")
    print(f"   • Rows: {len(sentiment_df)}")
    print(f"   • Sample sentiment scores: {sentiment_df['textblob_polarity'].head(3).tolist()}")
else:
    print("\n❌ sentiment_df not found in any path.")
    print("💡 Please run the sentiment analysis first or check the file path.")
    print("📁 Current directory:", os.getcwd())
    print("📁 Files in current directory:", [f for f in os.listdir('.') if os.path.isfile(f)])
    
    # Create empty feature_df to avoid further errors
    feature_df = pd.DataFrame()
    print("📝 Created empty feature_df for demonstration")


6. 🛠️ FEATURE ENGINEERING


🚀 Starting feature engineering pipeline...
❌ Not found: notebook/data/processed/sentiment_analysis_results.csv
❌ Not found: ../notebook/data/processed/sentiment_analysis_results.csv
❌ Not found: ../../notebook/data/processed/sentiment_analysis_results.csv
✅ sentiment_df loaded from: ./data/processed/sentiment_analysis_results.csv
📊 Original dataset shape: (12, 15)
✅ Feature DataFrame initialized

🔍 SENTIMENT_DF INFO:
   • Columns: ['textblob_polarity', 'textblob_subjectivity', 'vader_compound', 'vader_positive', 'vader_negative', 'vader_neutral', 'word_count', 'char_count', 'avg_word_length', 'sentiment_category', 'date', 'ticker', 'headline', 'publisher', 'original_sentiment']
   • Rows: 12
   • Sample sentiment scores: [0.0, 0.0, 0.5]


In [2]:
## 📝 TEXT-BASED FEATURES
print("\n1. 📝 TEXT-BASED FEATURES")
print("-" * 30)

# Sentiment intensity features
feature_df['sentiment_intensity'] = np.abs(feature_df['textblob_polarity'])
feature_df['vader_intensity'] = np.abs(feature_df['vader_compound'])
feature_df['sentiment_confidence'] = (feature_df['sentiment_intensity'] + feature_df['vader_intensity']) / 2

# Sentiment ratio features
feature_df['positive_ratio'] = feature_df['vader_positive'] / (feature_df['vader_positive'] + feature_df['vader_negative'] + 1e-8)
feature_df['negative_ratio'] = feature_df['vader_negative'] / (feature_df['vader_positive'] + feature_df['vader_negative'] + 1e-8)
feature_df['sentiment_balance'] = feature_df['vader_positive'] - feature_df['vader_negative']

print("✅ Text-based features created")
print(f"📊 Current dataset shape: {feature_df.shape}")


1. 📝 TEXT-BASED FEATURES
------------------------------
✅ Text-based features created
📊 Current dataset shape: (12, 21)


In [3]:
## 📝 TEXT-BASED FEATURES
print("\n1. 📝 TEXT-BASED FEATURES")
print("-" * 30)

# Sentiment intensity features
feature_df['sentiment_intensity'] = np.abs(feature_df['textblob_polarity'])
feature_df['vader_intensity'] = np.abs(feature_df['vader_compound'])
feature_df['sentiment_confidence'] = (feature_df['sentiment_intensity'] + feature_df['vader_intensity']) / 2

# Sentiment ratio features
feature_df['positive_ratio'] = feature_df['vader_positive'] / (feature_df['vader_positive'] + feature_df['vader_negative'] + 1e-8)
feature_df['negative_ratio'] = feature_df['vader_negative'] / (feature_df['vader_positive'] + feature_df['vader_negative'] + 1e-8)
feature_df['sentiment_balance'] = feature_df['vader_positive'] - feature_df['vader_negative']

print("✅ Text-based features created")


1. 📝 TEXT-BASED FEATURES
------------------------------
✅ Text-based features created


In [4]:
## 🔤 TEXT COMPLEXITY FEATURES
print("\n2. 🔤 TEXT COMPLEXITY FEATURES")
print("-" * 30)

# Text complexity features
feature_df['title_length'] = feature_df['headline'].str.len()
feature_df['word_density'] = feature_df['word_count'] / (feature_df['char_count'] + 1)
feature_df['exclamation_present'] = feature_df['headline'].str.contains('!').astype(int)
feature_df['question_present'] = feature_df['headline'].str.contains('\?').astype(int)
feature_df['has_colon'] = feature_df['headline'].str.contains(':').astype(int)

# Keyword-based features
positive_keywords = ['solid', 'strong', 'growth', 'positive', 'good', 'bullish', 'gain', 'profit']
negative_keywords = ['challenging', 'mixed', 'weak', 'negative', 'bad', 'bearish', 'loss', 'drop']

feature_df['positive_keyword_count'] = feature_df['headline'].apply(
    lambda x: sum(1 for word in positive_keywords if word in str(x).lower()))
feature_df['negative_keyword_count'] = feature_df['headline'].apply(
    lambda x: sum(1 for word in negative_keywords if word in str(x).lower()))
feature_df['net_keyword_sentiment'] = feature_df['positive_keyword_count'] - feature_df['negative_keyword_count']

print("✅ Text complexity features created")
print(f"📊 Current dataset shape: {feature_df.shape}")


2. 🔤 TEXT COMPLEXITY FEATURES
------------------------------
✅ Text complexity features created
📊 Current dataset shape: (12, 29)


In [5]:
## 🏢 COMPANY & TIME-BASED FEATURES
print("\n3. 🏢 COMPANY & TIME-BASED FEATURES")
print("-" * 30)

# Company encoding
le_company = LabelEncoder()
feature_df['company_encoded'] = le_company.fit_transform(feature_df['ticker'])

# Date features
feature_df['date'] = pd.to_datetime(feature_df['date'])
feature_df['year'] = feature_df['date'].dt.year
feature_df['month'] = feature_df['date'].dt.month
feature_df['quarter'] = feature_df['date'].dt.quarter
feature_df['day_of_week'] = feature_df['date'].dt.dayofweek
feature_df['is_weekend'] = (feature_df['day_of_week'] >= 5).astype(int)

print("✅ Company and time features created")
print(f"📊 Current dataset shape: {feature_df.shape}")


3. 🏢 COMPANY & TIME-BASED FEATURES
------------------------------
✅ Company and time features created
📊 Current dataset shape: (12, 35)


In [6]:
## 📊 COMPANY SENTIMENT AGGREGATES
print("\n4. 📊 COMPANY SENTIMENT AGGREGATES")
print("-" * 30)

# Company sentiment aggregates
company_stats = feature_df.groupby('ticker').agg({
    'textblob_polarity': ['mean', 'std', 'min', 'max'],
    'vader_compound': ['mean', 'std']
}).round(4)

company_stats.columns = ['company_avg_polarity', 'company_std_polarity', 
                        'company_min_polarity', 'company_max_polarity',
                        'company_avg_vader', 'company_std_vader']

feature_df = feature_df.merge(company_stats, on='ticker', how='left')

# Relative sentiment features
feature_df['polarity_vs_company_avg'] = feature_df['textblob_polarity'] - feature_df['company_avg_polarity']
feature_df['vader_vs_company_avg'] = feature_df['vader_compound'] - feature_df['company_avg_vader']
feature_df['sentiment_zscore'] = (feature_df['textblob_polarity'] - feature_df['company_avg_polarity']) / (feature_df['company_std_polarity'] + 1e-8)

print("✅ Company sentiment aggregates created")
print(f"📊 Current dataset shape: {feature_df.shape}")


4. 📊 COMPANY SENTIMENT AGGREGATES
------------------------------
✅ Company sentiment aggregates created
📊 Current dataset shape: (12, 44)
